## **[ Produced by ]**

[Eunchan Lee of KNU-BrainAI](https://github.com/purang2)


**[ 목차 ]**


---



### Requirements 설치 

In [1]:
!pip install transformers 
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 4.1 MB/s 
     |████████████████████████████████| 6.6 MB 33.8 MB/s 
     |████████████████████████████████| 596 kB 38.4 MB/s 
     |████████████████████████████████| 101 kB 10.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 117 kB 4.3 MB/s 


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM 

## Model Loading

Huggingface/transformers 패키지를 통해 토크나이저와 모델을 load 

Model size: 11.5 GB (큰 것 같다 -> 16GB 권장, 고용량의 RAM 필요)
- KoGPT6B-ryan1.5b는 32GB GPU RAM을 최소로 권장 
- KoGPT6B-ryan1.5b-float16는 16GB GPU RAM을 권장



In [3]:
tokenizer = AutoTokenizer.from_pretrained(
    'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16', # or float32 version: revision=KoGPT6B-ryan1.5b
    bos_token = '[BOS]', eos_token='[EOS]', unk_token = '[UNK]', pad_token='[PAD]', mask_token = '[MASK]'
)

model = AutoModelForCausalLM.from_pretrained(
    'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',# or float32 version: revision=KoGPT6B-ryan1.5b
  pad_token_id=tokenizer.eos_token_id,
  torch_dtype='auto', low_cpu_mem_usage=True
).to(device='cuda', non_blocking=True)

_  = model.eval()

Downloading:   0%|          | 0.00/252 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.40M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/839 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/11.5G [00:00<?, ?B/s]

Zero-finetune 상태에서도 아래와 같이 잘 생성해낸다. 

In [4]:
prompt = '인간처럼 생각하고, 행동하는 \'지능\'을 통해 인류가 이제까지 풀지 못했던'

with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=64)
  generated = tokenizer.batch_decode(gen_tokens)[0]

print(generated)

인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 최대의 난제를 해결할 수 있을 것이다. '지적 설계론'(Tertiary Design)... '과학적 창조론'은 '지적 설계론'으로 바뀌어야 한다. (


## Examples 

GPT-3는 Train이 없거나 적은 양으로도 태스크를 수행할 수 있는 Few-shot, Zero-shot 환경에서도 매우 우수한 성능을 보여주었다.  

이 예제에서는 GPT-3를 모티브로 개발된 **Kakaobrain/KoGPT**를 통해 한국어 기반 사설 자연어처리 데이터셋을 통해 Few-shot 성능을 검증해본다.

In [6]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 4.2 MB/s 
     |████████████████████████████████| 212 kB 58.0 MB/s 
     |████████████████████████████████| 1.1 MB 58.5 MB/s 
     |████████████████████████████████| 140 kB 53.6 MB/s 
     |████████████████████████████████| 127 kB 63.1 MB/s 
     |████████████████████████████████| 144 kB 53.8 MB/s 
     |████████████████████████████████| 94 kB 3.8 MB/s 
     |████████████████████████████████| 271 kB 53.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


## NSMC 데이터셋 loading

NSMC 데이터셋은 Naver sentiment movie corpus, 즉 네이버 영화의 리뷰 text와 그것이 positive(1) or negative(0)인지 나타내는 label의 pair로 구성된 데이터셋이다. 분포는 아래와 같다

DatasetDict({  
    train: Dataset({  
        features: ['id', 'document', 'label'],  
        num_rows: 150000  
    })  
    test: Dataset({  
        features: ['id', 'document', 'label'],  
        num_rows: 50000  
    })  
})  

In [7]:
from datasets import load_dataset

dataset = load_dataset("nsmc")

Using custom data configuration default


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/150000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset nsmc downloaded and prepared to /root/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})

In [18]:
dataset['test'][60]['document']

'콩콩~~~~ㅋㄲㅈㅁ'

In [21]:
dataset['train'][10]['document']

'걍인피니트가짱이다.진짜짱이다♥'

## Few-shot learning을 위한 Prompt 생성

KoGPT이므로 Task description을 한글로 만들어야 하는가??
우선 한글로 접근해본다.

In [48]:
prompt = '' #few-shot prompt 

#task description of prompt
prompt+= '영화 리뷰가 긍정인지 부정인지 분류하기:\n\n'

#Add K examples in the context window (Few-shot Learning), train set 50개 집어 넣기
#format : "솰라솰라블라블라(document) => 0(label)"


#this is for 0->'negative' or 1->'positive'
label_to_txt = ['부정','긍정']

for i in range(70,120):
  prompt+=f"{dataset['train'][i]['document']} => {label_to_txt[dataset['train'][i]['label']]}\n\n"


print(prompt)

영화 리뷰가 긍정인지 부정인지 분류하기:

아 일본영화 다이런건가?? 유치하다 => 부정

이틀만에 다 봤어요 재밌어요 근데 차 안에 물건 넣어 조작하려고 하면 차 안이 열려있다던지 집 안이 활짝 열려서 아무나 들어간다던가 문자를 조작하려고하면 비번이 안 걸려있고 ㅋㅋㅋ 그런 건 억지스러웠는데 그래도 내용 자체는 좋았어요 => 긍정

졸작 => 부정

재밋네요 달팽이가 빨라서 더 재밌었어요 => 긍정

어설픈 전개 어이없는 결말 => 부정

부패한 로마노프 왕조를 기리는 뭣같은 영화... 온몸으로 항거했던 러시아 민중들이 그저 폭도냐 => 부정

내용전개는 무난한 편이였구 잘 보았습니다 ^^ => 긍정

매우 실망..... => 부정

한국영화 흥행코드: 갈등-갈등-계~에속 갈등-화해-감동- 평점 10점 남발- 흥행 뻔하지 뭐... => 부정

아햏햏 아햏햏 아햏햏. => 긍정

뭐냐..시작하고 3분만에 나왔다. 리플릿 사진 보며 불안하더니만.. => 부정

단연 최고라고 할수있지 => 긍정

감독이 럼먹고 영화를 만들었나보다.. 관객에게 뭘 말하는지도 모르겠고, 엉망진창 개진창이다. => 부정

이건 뭐냐? 우뢰매냐? ;;; => 부정

정말쓰레기영화입니다 => 부정

진정 위대한 영화 최고임 => 긍정

별루 였다.. => 부정

내일이 기대되는 ` => 긍정

근데 조미가 막문위 좋아한건가요?? => 긍정

ㅋㅋㅋ 진짜 골깜..ㅋㅋ 눈 부라릴때 쓰러짐..ㅋㅋ => 긍정

성룡영화중 최악인듯 ㅋㅋ => 부정

골때리네ㅋㅋㅋㅋ 걸스데이 이혜리 잘 되라! => 긍정

서기가이뻐서 => 긍정

완전 재밌어요ㅋㅋㅋㅋㅋ백인공주귀여움ㅋㅋㅋㅋㅋㅋ => 긍정

인상적인 영화였다 => 긍정

어내스트와 셀레스틴 완전 강추에요~ 정말 재밌습니다^^ => 긍정

재미있는영화입니다. => 긍정

클라라볼라고화신본거아닌데 => 부정

진짜 보면서 너무 슬펐던 영화다 => 긍정

설정이 재밌고 새로운 에피소드 내에서 메인 스토리도 차차 나오는게 재밌음 => 긍정

신카이 마코토의 

## NSMC Test Dataset KoGPT Few-shot Learning 실험 진행하기

아래 5개의 코드 블록은 테스트이다.

In [49]:

#for i in range(num(test)):
for i in range(1):
  prompts = prompt + f"{dataset['test'][i]['document']} => " 
  with torch.no_grad():
    tokens = tokenizer.encode(prompts, return_tensors='pt').to(device='cuda', non_blocking=True)
    gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=50)
    generated = tokenizer.batch_decode(gen_tokens)[0]
    



Input length of input_ids is 849, but ``max_length`` is set to 50. This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.


In [50]:
dataset['test'][0]['document']

'굳 ㅋ'

In [51]:
generated

"영화 리뷰가 긍정인지 부정인지 분류하기:\n\n아 일본영화 다이런건가?? 유치하다 => 부정\n\n이틀만에 다 봤어요 재밌어요 근데 차 안에 물건 넣어 조작하려고 하면 차 안이 열려있다던지 집 안이 활짝 열려서 아무나 들어간다던가 문자를 조작하려고하면 비번이 안 걸려있고 ㅋㅋㅋ 그런 건 억지스러웠는데 그래도 내용 자체는 좋았어요 => 긍정\n\n졸작 => 부정\n\n재밋네요 달팽이가 빨라서 더 재밌었어요 => 긍정\n\n어설픈 전개 어이없는 결말 => 부정\n\n부패한 로마노프 왕조를 기리는 뭣같은 영화... 온몸으로 항거했던 러시아 민중들이 그저 폭도냐 => 부정\n\n내용전개는 무난한 편이였구 잘 보았습니다 ^^ => 긍정\n\n매우 실망..... => 부정\n\n한국영화 흥행코드: 갈등-갈등-계~에속 갈등-화해-감동- 평점 10점 남발- 흥행 뻔하지 뭐... => 부정\n\n아햏햏 아햏햏 아햏햏. => 긍정\n\n뭐냐..시작하고 3분만에 나왔다. 리플릿 사진 보며 불안하더니만.. => 부정\n\n단연 최고라고 할수있지 => 긍정\n\n감독이 럼먹고 영화를 만들었나보다.. 관객에게 뭘 말하는지도 모르겠고, 엉망진창 개진창이다. => 부정\n\n이건 뭐냐? 우뢰매냐? ;;; => 부정\n\n정말쓰레기영화입니다 => 부정\n\n진정 위대한 영화 최고임 => 긍정\n\n별루 였다.. => 부정\n\n내일이 기대되는 ` => 긍정\n\n근데 조미가 막문위 좋아한건가요?? => 긍정\n\nㅋㅋㅋ 진짜 골깜..ㅋㅋ 눈 부라릴때 쓰러짐..ㅋㅋ => 긍정\n\n성룡영화중 최악인듯 ㅋㅋ => 부정\n\n골때리네ㅋㅋㅋㅋ 걸스데이 이혜리 잘 되라! => 긍정\n\n서기가이뻐서 => 긍정\n\n완전 재밌어요ㅋㅋㅋㅋㅋ백인공주귀여움ㅋㅋㅋㅋㅋㅋ => 긍정\n\n인상적인 영화였다 => 긍정\n\n어내스트와 셀레스틴 완전 강추에요~ 정말 재밌습니다^^ => 긍정\n\n재미있는영화입니다. => 긍정\n\n클라라볼라고화신본거아닌데 => 부정\n\n진짜 보면서 너무 슬펐던 영화다 => 긍정

In [54]:
generated[-2:] #will be label.

'긍정'

In [56]:
label_to_txt.index('긍정')

1

In [64]:
james = 3545 / 5000 * 100
james = round(james, 3)
james 

70.9

여기부터 NSMC Few-shot 검증 실험!😁

하나의 캡션 생성(model.generate)에 많은 시간이 걸린다: 

In [76]:
CORRECT = 0 #맞은 개수 counting -> accuracy 계산에 활용
acc = 0

for i in range(len(dataset['test'])):
  prompts = prompt + f"{dataset['test'][i]['document']} => " 
  with torch.no_grad():
    tokens = tokenizer.encode(prompts, return_tensors='pt').to(device='cuda', non_blocking=True)
    gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=10)
    generated = tokenizer.batch_decode(gen_tokens)[0]
  predicted_label = generated[-2:]
  print(i)  
  try:
    predicted = label_to_txt.index(predicted_label)
    
  except: 
    predicted = ''

  if predicted == dataset['test'][i]['label']: CORRECT += 1
  if i+1 % 50 == 0 and i != 0: 
    acc = round(CORRECT / i * 100, 3)
    print(f'== Accuracy : {acc} ==== {CORRECT} / {i} ') 

Input length of input_ids is 849, but ``max_length`` is set to 10. This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.


RuntimeError: ignored